In [1]:
import cv2
import numpy as np
import rasterio
import matplotlib.pyplot as plt

In [2]:
def crop_ndvi_image(ndvi_image_path, output_cropped_image_path):
    # Lecture de l'image NDVI
    with rasterio.open(ndvi_image_path) as src:
        ndvi = src.read(1)
    
    # Normalisation de l'image pour OpenCV
    ndvi_normalized = cv2.normalize(ndvi, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    # Détection des contours avec OpenCV
    _, thresholded = cv2.threshold(ndvi_normalized, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Trouver le contour le plus grand
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Découpe de l'image
    cropped_ndvi = ndvi[y:y+h, x:x+w]
    
    # Affichage de l'image découpée
    plt.imshow(cropped_ndvi, cmap='RdYlGn')
    plt.colorbar()
    plt.title('NDVI découpé')
    plt.show()
    
    # Sauvegarde de l'image découpée
    with rasterio.open(
        output_cropped_image_path, 
        'w', 
        driver='GTiff', 
        height=cropped_ndvi.shape[0],
        width=cropped_ndvi.shape[1],
        count=1,
        dtype='float32',
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(cropped_ndvi, 1)
    
    print(f"Image NDVI découpée sauvegardée dans {output_cropped_image_path}")

In [ ]:
if __name__ == "__main__":
    crop_ndvi_image('output_ndvi.tif', 'output_ndvi_cropped.tif')